In [2]:
# Connect to local DB
import os
import mysql.connector
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = "127.0.0.1"
db_name = "coursera_db"
engine = mysql.connector.connect(user=db_user, password=db_password, host=db_host, database=db_name)

cur = engine.cursor()

Several key statments to help retrieve, manipulate, and analyze data efficiently. 

### ORDER BY - sort results
Sorts data in <b>ascending</b> (ASC) or <b>descending</b> (DESC) order.
``` 
SELECT name, age FROM users ORDER BY age DESC; 
```

In [6]:
# Use order by
sql = "SELECT F_NAME, L_NAME, DEP_ID FROM employees ORDER BY DEP_ID"
cur.execute(sql)
rows = cur.fetchall()

print(type(rows))

for row in rows:
    print(row)

<class 'list'>
('John', 'Thomas', '2')
('Ahmed', 'Hussain', '2')
('Nancy', 'Allen', '2')
('Alice', 'James', '5')
('Steve', 'Wells', '5')
('Santosh', 'Kumar', '5')
('Ann', 'Jacob', '5')
('Mary', 'Thomas', '7')
('Bharath', 'Gupta', '7')
('Andrea', 'Jones', '7')


In [8]:
# Another order by 
sql = "SELECT F_NAME, L_NAME, DEP_ID FROM employees ORDER BY DEP_ID DESC, L_NAME DESC"
cur.execute(sql)
rows = cur.fetchall()

print(type(rows))

for row in rows:
    print(row)

<class 'list'>
('Mary', 'Thomas', '7')
('Andrea', 'Jones', '7')
('Bharath', 'Gupta', '7')
('Steve', 'Wells', '5')
('Santosh', 'Kumar', '5')
('Alice', 'James', '5')
('Ann', 'Jacob', '5')
('John', 'Thomas', '2')
('Ahmed', 'Hussain', '2')
('Nancy', 'Allen', '2')


### GROUP BY - aggregate data
Used with aggregate functions (`COUNT()`, `SUM()`, `AVG()`, etc.)
```
SELECT department, COUNT(*) AS employee_count
FROM employees
GROUP BY department;
```

In [9]:
# Using Group By
sql = "SELECT DEP_ID, COUNT(*) FROM employees GROUP BY DEP_ID"
cur.execute(sql)
rows = cur.fetchall()

for row in rows:
    print(row)

('2', 3)
('5', 4)
('7', 3)


In [11]:
# Another Group By
sql = "SELECT DEP_ID, COUNT(*), AVG(SALARY) FROM employees GROUP BY DEP_ID"
cur.execute(sql)
rows = cur.fetchall()

for row in rows:
    print(row)

('2', 3, Decimal('86666.666667'))
('5', 4, Decimal('65000.000000'))
('7', 3, Decimal('66666.666667'))


### HAVING - filter aggregate data
`HAVING` is uded to filter <b>aggregated</b> results from `GROUP BY`, whereas `WHERE` filters <b>individual rows</b>.
```
SELECT departments, COUNT(*) as employee_count
FROM employees
GROUP BY department
HAVING COUNT(*) > 5;
```

In [16]:
# Using Having
sql = """SELECT DEP_ID, COUNT(*) AS 'num_employess', AVG(SALARY) AS 'avg_salary' 
FROM employees 
GROUP BY DEP_ID 
HAVING COUNT(*) < 4 
ORDER BY avg_salary"""
cur.execute(sql)
rows = cur.fetchall()

for row in rows:
    print(row)

('7', 3, Decimal('66666.666667'))
('2', 3, Decimal('86666.666667'))


### UNION - combine query results
Merges results from multiple SELECT queries while removing duplicates.
```
SELECT name FROM customers
UNION
SELECT name FROM suppliers;
```

## CASE - conditional logic in SQL
Works like an `IF-ELSE` statement in SQL.
```
SELECT name,
    CASE
        WHEN age < 18 THEN 'minor'
        WHEN age between 18 AND 65 THEN 'adult'
        ELSE 'senior'
    END AS age_group
FROM users;
```

### EXISTS - check if a subquery returns rows
```
SELECT name FROM users
WHERE EXISTS (SELECT * FROM orders WHERE users.id = orders.user_id);
```

In [ ]:
# Pull DB data to verify
cur.execute("SELECT * FROM medical_departments")
med_depart_data = cur.fetchall()

print(med_depart_data)

### What does `.commit` do?
`.commit()` is the method is database connection objects that finalizes changes made in a transcation and makes them permanent in the database.
<b>Without `.commit()`, the changes remain temporary and are lost when the connection is closed.</b>

### Why is `.commit` necessary?
When you modify data in MySQL, it does not immediately write those changes permanently. Instead, <b>it stores them in a temporary state (transction buffer)</b> until either:
- `.commit()` is called -> <b>makes the changes permanent</b>
- `.rollback()` is called -> <b>reverts changes</b>
- The connection closes <b>without committing</b> -> <b>changes are lost</b>

### Why can you still query uncommitted data?
Even if you havent committed, you can still <b>see uncommitted changes within the same connection</b> because:
- <b>MySQL supports transactions</b>, meaning the changes are visible only within the active session.
- Myour changes exist in a <b>temporary transaction buffer</b>, not yet written permanently to disk.

### How `.commit()` works internally?
When you call `.commit()` MySQL:
1. <b>Writes the transaction log</b> -> ensures durability.
2. <b>Releases locks</b> -> Allows other users to see the changes.
3. <b>Flushes changes to disk</b> -> Now permanently stored in the database.

In [53]:
engine.commit()

In [ ]:

cur.close()
engine.close()